0) Install & imports

In [5]:
!pip -q install osmnx==1.9.3 geopandas==0.14.4 shapely==2.0.4 \
                networkx==3.2.1 pandas==2.2.2 gtfs-kit==6.1 \
                pyproj==3.6.1 pyrosm==0.6.2 scipy==1.11.4

import os, requests
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.validation import make_valid
from pyrosm import OSM
from scipy.spatial import cKDTree

ox.settings.use_cache = False
ox.settings.log_console = True
ox.settings.log_file = False

# Parameters
WALK_MPS = 1.0          # walking speed (m/s)
MAX_WALK_MIN = 30       # drop outliers >= 30 min

# Paths (EDIT THESE TWO)
DISTRICTS_PATH = "/Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/files/berliner-bezirke.geojson"
GTFS_ZIP       = "/Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/files/GTFS.zip"

# PBF will be downloaded here if missing
PBF_PATH       = "berlin-latest.osm.pbf"

1) Districts (local GeoJSON) → AOI

In [6]:
# Read local Bezirke GeoJSON — do NOT transform yet
districts = gpd.read_file(DISTRICTS_PATH)

# Ensure CRS tag
if districts.crs is None:
    districts = districts.set_crs(4326)  # assign, don't transform

# Clean geometries
districts = districts[~districts.geometry.is_empty & districts.geometry.notna()].copy()
districts["geometry"] = districts.geometry.buffer(0)
districts["geometry"] = districts.geometry.map(make_valid)

# Standardize name column
name_col = next((c for c in ["district","name","BEZIRKSNAME","GEN","bezirk","spatial_alias"]
                 if c in districts.columns), None)
if name_col is None:
    raise ValueError("District name column not found.")
districts = districts.rename(columns={name_col: "district"})

# AOI polygon
aoi = districts.unary_union
print("Districts:", len(districts), "| CRS:", districts.crs)

Districts: 12 | CRS: EPSG:4326


2) Ensure PBF locally, then build walk graph (pyrosm → osmnx) + prune invalid nodes

In [7]:
# Download PBF once if missing
if not os.path.exists(PBF_PATH):
    print("Downloading berlin-latest.osm.pbf (~100 MB)…")
    url = "https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf"
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(PBF_PATH, "wb") as f:
            for ch in r.iter_content(chunk_size=8192):
                f.write(ch)
    print("Done:", PBF_PATH)

# Build walk network from local PBF
osm = OSM(PBF_PATH)
out = osm.get_network(network_type="walking", nodes=True)

# Detect which is edges/nodes
if isinstance(out, tuple) and len(out) == 2:
    gdf_a, gdf_b = out
    if {"u","v"}.issubset(gdf_a.columns) or {"from_id","to_id"}.issubset(gdf_a.columns):
        edges, nodes = gdf_a, gdf_b
    else:
        edges, nodes = gdf_b, gdf_a
else:
    raise RuntimeError("pyrosm did not return (edges, nodes).")

# Normalize edges to have 'u','v'
if not {"u","v"}.issubset(edges.columns):
    if {"from_id","to_id"}.issubset(edges.columns):
        edges = edges.rename(columns={"from_id":"u","to_id":"v"})
    else:
        raise ValueError("Edges missing 'u','v' (or 'from_id','to_id').")

# Nodes need ID index and x/y
node_id_col = "id" if "id" in nodes.columns else ("node_id" if "node_id" in nodes.columns else None)
if node_id_col is None:
    raise ValueError("Nodes have no id column ('id' or 'node_id').")
nodes = nodes.set_index(node_id_col)
if not {"x","y"}.issubset(nodes.columns):
    nodes["x"] = nodes.geometry.x
    nodes["y"] = nodes.geometry.y

# MultiIndex on edges (u,v,key)
if "key" not in edges.columns:
    edges["key"] = edges.groupby(["u","v"]).cumcount()
edges = edges.set_index(["u","v","key"], drop=True).sort_index()

# Build graph, project to metric CRS, add time weights
G  = ox.graph_from_gdfs(nodes, edges)
Gp = ox.project_graph(G)
nodes_gdf, edges_gdf = ox.graph_to_gdfs(Gp)

for _, _, k, d in Gp.edges(keys=True, data=True):
    d["walk_time_s"] = d.get("length", 0) / WALK_MPS

# Reproject districts to graph CRS
districts = districts.to_crs(nodes_gdf.crs)

# PRUNE: keep only nodes with finite x/y and largest connected component
mask_nodes = np.isfinite(nodes_gdf["x"].to_numpy()) & np.isfinite(nodes_gdf["y"].to_numpy())
if mask_nodes.sum() != len(nodes_gdf):
    Gp = Gp.subgraph(nodes_gdf.index[mask_nodes]).copy()

Gp = Gp.subgraph(max(nx.connected_components(Gp.to_undirected()), key=len)).copy()
nodes_gdf, edges_gdf = ox.graph_to_gdfs(Gp)

print(f"Graph: {len(nodes_gdf):,} nodes, {len(edges_gdf):,} edges | CRS: {nodes_gdf.crs}")

Done: berlin-latest.osm.pbf
Graph: 1,188,705 nodes, 1,417,030 edges | CRS: EPSG:32633


3) Buildings from PBF → representative points → tag district → KDTree snap (cached)

In [9]:
CACHE_GPKG = "buildings_points_snapped.gpkg"
CACHE_LAYER = "bld_points_snapped"

try:
    bld = gpd.read_file(CACHE_GPKG, layer=CACHE_LAYER)
    if getattr(bld, "crs", None) != nodes_gdf.crs:
        bld = bld.to_crs(nodes_gdf.crs)
    print(f"Loaded cached buildings: {len(bld):,}")
except Exception:
    bld_poly = osm.get_buildings()
    if bld_poly is None or bld_poly.empty:
        raise RuntimeError("No buildings returned from PBF.")

    bld_poly = bld_poly.to_crs(nodes_gdf.crs)
    bld_poly = bld_poly[~bld_poly.geometry.is_empty & bld_poly.geometry.notna()].copy()
    bld_poly["geometry"] = bld_poly.geometry.map(make_valid)
    bld_poly = bld_poly[bld_poly.geometry.geom_type.isin(["Polygon","MultiPolygon"])]
    bld_poly = gpd.clip(bld_poly, districts.unary_union)

    # Representative points
    bld = bld_poly[["geometry"]].reset_index(drop=True)
    bld["geometry"] = bld.geometry.representative_point()
    bld = gpd.GeoDataFrame(bld, geometry="geometry", crs=nodes_gdf.crs)
    bld["bid"] = np.arange(len(bld))

    # Tag district
    bld = gpd.sjoin(bld, districts[["district","geometry"]], predicate="within", how="inner").drop(columns=["index_right"])

    # KDTree snap to graph nodes (robust)
    NX = nodes_gdf["x"].to_numpy(dtype=float)
    NY = nodes_gdf["y"].to_numpy(dtype=float)
    node_ids = nodes_gdf.index.to_numpy()
    mask_nodes = np.isfinite(NX) & np.isfinite(NY)
    tree = cKDTree(np.column_stack([NX[mask_nodes], NY[mask_nodes]]))
    node_ids = node_ids[mask_nodes]

    PX = bld.geometry.x.to_numpy(dtype=float)
    PY = bld.geometry.y.to_numpy(dtype=float)
    mask_pts = np.isfinite(PX) & np.isfinite(PY)
    ii = tree.query(np.column_stack([PX[mask_pts], PY[mask_pts]]), k=1)[1]
    bld.loc[mask_pts, "node_id"] = node_ids[ii]

    bld = bld.dropna(subset=["node_id"])

    # Cache
    try:
        bld.to_file(CACHE_GPKG, layer=CACHE_LAYER, driver="GPKG")
        print(f"Cached buildings → {CACHE_GPKG}")
    except Exception as e:
        print("Warning: could not write cache:", e)

print(
    f"Buildings: {len(bld):,} | Districts: {bld['district'].nunique()} / 12 | "
    f"Unique nodes: {bld['node_id'].nunique():,}"
)
bld.head()

Cached buildings → buildings_points_snapped.gpkg
Buildings: 501,367 | Districts: 12 / 12 | Unique nodes: 215,226


,geometry,bid,district,node_id
0,POINT (407875.203 5799689.256),0,Treptow-Köpenick,92141861.0
1,POINT (407860.064 5799691.236),1,Treptow-Köpenick,92141861.0
2,POINT (407854.588 5799708.047),2,Treptow-Köpenick,92141861.0
3,POINT (407807.128 5799715.723),3,Treptow-Köpenick,386025753.0
4,POINT (407773.364 5799717.686),4,Treptow-Köpenick,92141858.0


4) GTFS → map route_type → modes → KDTree snap to graph nodes

In [10]:
import gtfs_kit as gk

feed = gk.read_feed(GTFS_ZIP, dist_units="km")

# Robust route_type → mode
def map_route_type_to_mode(rt):
    if pd.isna(rt): return None
    try: rt = int(rt)
    except: return None
    # Standard
    if rt == 0:  return "Tram"
    if rt == 1:  return "U-Bahn"
    if rt == 2:  return "S-Bahn"
    if rt == 3:  return "Bus"
    # Common extensions
    if rt in {401}:          return "U-Bahn"   # metro
    if rt in {100, 402}:     return "S-Bahn"   # rail
    if 700 <= rt < 800 or rt in {109}: return "Bus"
    if rt in {900, 404, 405}: return "Tram"
    return None

trip_modes = feed.trips[["trip_id","route_id"]].merge(
    feed.routes[["route_id","route_type","route_short_name","route_long_name"]],
    on="route_id", how="left"
)
trip_modes["mode"] = trip_modes["route_type"].map(map_route_type_to_mode)

stops_served = feed.stop_times[["stop_id","trip_id"]].merge(trip_modes, on="trip_id", how="left")
stops_served = stops_served.dropna(subset=["mode"])

stop_mode_pairs = (stops_served
                   .groupby(["stop_id","mode"], as_index=False)
                   .size()
                   .rename(columns={"size":"n"}))

# Clean GTFS stops and reproject to graph CRS
stops = (feed.stops[["stop_id","stop_lat","stop_lon"]]
         .dropna(subset=["stop_lat","stop_lon"])
         .copy())
stops = stops[
    np.isfinite(stops.stop_lat) & np.isfinite(stops.stop_lon) &
    stops.stop_lat.between(-90, 90) & stops.stop_lon.between(-180, 180)
]
stops["geometry"] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat, crs=4326)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326).to_crs(nodes_gdf.crs)
stops = stops[stops.geometry.notna() & ~stops.geometry.is_empty].copy()
stops = gpd.clip(stops, districts.unary_union)

# Merge modes + geometry
stops_modes = stop_mode_pairs.merge(stops[["stop_id","geometry"]], on="stop_id", how="inner")
stops_modes = gpd.GeoDataFrame(stops_modes, geometry="geometry", crs=nodes_gdf.crs)

# KDTree snap (replace ox.nearest_nodes to avoid NaN issues)
NX = nodes_gdf["x"].to_numpy(dtype=float)
NY = nodes_gdf["y"].to_numpy(dtype=float)
node_ids = nodes_gdf.index.to_numpy()
mask_nodes = np.isfinite(NX) & np.isfinite(NY)
tree = cKDTree(np.column_stack([NX[mask_nodes], NY[mask_nodes]]))
node_ids = node_ids[mask_nodes]

PX = stops_modes.geometry.x.to_numpy(dtype=float)
PY = stops_modes.geometry.y.to_numpy(dtype=float)
mask_pts = np.isfinite(PX) & np.isfinite(PY)
ii = tree.query(np.column_stack([PX[mask_pts], PY[mask_pts]]), k=1)[1]
stops_modes.loc[mask_pts, "node_id"] = node_ids[ii]

print("Stops per mode:\n", stops_modes["mode"].value_counts())
print("Total snapped stops:", len(stops_modes))

Stops per mode:
 mode
Bus       6561
Tram       840
S-Bahn      77
Name: count, dtype: int64
Total snapped stops: 7478


5) Multi-source Dijkstra per mode → per-building walk time → average by district → 12×4 table

In [11]:
def node_times_to_sources(G, source_nodes):
    """Shortest walk_time_s to any source node for all nodes reachable."""
    if not source_nodes:
        return {}
    super_src = -1
    if super_src in G: G.remove_node(super_src)
    G.add_node(super_src)
    for n in source_nodes:
        G.add_edge(super_src, n, walk_time_s=0)
    return nx.single_source_dijkstra_path_length(G, super_src, weight="walk_time_s")

modes = ["S-Bahn", "U-Bahn", "Tram", "Bus"]
per_mode = {}

for m in modes:
    mode_nodes = stops_modes.loc[stops_modes["mode"] == m, "node_id"].dropna().unique().tolist()
    if not mode_nodes:
        per_mode[m] = pd.DataFrame({"district": [], m: []})
        print(f"Warning: no stop nodes for mode '{m}'")
        continue

    times = node_times_to_sources(Gp, mode_nodes)

    df = bld[["district","node_id"]].copy()
    df["walk_min"] = df["node_id"].map(lambda nid: (times.get(nid, np.nan) / 60.0))
    df = df.dropna(subset=["walk_min"])
    df = df[df["walk_min"] < MAX_WALK_MIN]  # filter outliers >= 30

    if df.empty:
        per_mode[m] = pd.DataFrame({"district": [], m: []})
        print(f"Warning: all values filtered for mode '{m}'")
        continue

    per_mode[m] = (df.groupby("district")["walk_min"]
                     .mean()
                     .reset_index()
                     .rename(columns={"walk_min": m}))

# Assemble wide table with all 12 districts
table = districts[["district"]].drop_duplicates().sort_values("district")
for m in modes:
    table = table.merge(per_mode[m], on="district", how="left")

table[modes] = table[modes].round(2)
table

,district,S-Bahn,U-Bahn,Tram,Bus
0,Charlottenburg-Wilmersdorf,20.85,NaN,27.36,9.10
1,Friedrichshain-Kreuzberg,22.14,NaN,10.04,7.23
2,Lichtenberg,22.23,NaN,13.25,11.71
3,Marzahn-Hellersdorf,21.80,NaN,16.03,10.00
4,Mitte,15.91,NaN,16.48,8.01
5,Neukölln,NaN,NaN,28.09,8.41
6,Pankow,20.43,NaN,15.50,11.00
7,Reinickendorf,NaN,NaN,23.13,10.46
8,Spandau,20.61,NaN,NaN,9.57
9,Steglitz-Zehlendorf,0.25,NaN,NaN,8.86


In [12]:
# What route types and example route names do we have?
print("route_type counts:\n", feed.routes["route_type"].value_counts(dropna=False).sort_index(), "\n")
print("Sample U-like routes by short_name:\n",
      feed.routes[feed.routes["route_short_name"].str.startswith("U", na=False)]
      [["route_id","route_short_name","route_long_name","route_type"]].head(15))

route_type counts:
 route_type
2          2
3         81
100       52
106       18
109       50
400        9
700     1067
900       43
1000       8
Name: count, dtype: int64 

Sample U-like routes by short_name:
       route_id route_short_name route_long_name  route_type
425  17994_700               U2             NaN         700
441  17526_400               U9             NaN         400
442  17525_400               U8             NaN         400
443  17523_400               U7             NaN         400
444  17522_700               U7             NaN         700
445  17521_400               U6             NaN         400
446  17520_700               U6             NaN         700
447  17518_400               U5             NaN         400
448  17517_700               U5             NaN         700
449  17516_400               U4             NaN         400
450  17515_400               U3             NaN         400
451  17514_400               U2             NaN         400
452  17

In [13]:
import pandas as pd

def map_route_type_to_mode(rt):
    if pd.isna(rt): return None
    try: rt = int(rt)
    except: return None
    # Standard
    if rt == 0:  return "Tram"
    if rt == 1:  return "U-Bahn"
    if rt == 2:  return "S-Bahn"
    if rt == 3:  return "Bus"
    # Common extensions
    if rt in {401}:              return "U-Bahn"   # metro
    if rt in {100, 402}:         return "S-Bahn"   # rail
    if 700 <= rt < 800 or rt in {109}: return "Bus"
    if rt in {900, 404, 405}:    return "Tram"
    return None

# trips -> routes -> (mode_by_code + names)
routes = feed.routes[["route_id","route_type","route_short_name","route_long_name"]].copy()
routes["mode_code"] = routes["route_type"].map(map_route_type_to_mode)

# Name-based fallback/override for U-Bahn and Tram (Berlin specifics)
def name_mode(row):
    sn = row["route_short_name"]
    ln = row["route_long_name"]
    sn = sn.upper() if isinstance(sn, str) else ""
    ln = ln.upper() if isinstance(ln, str) else ""
    if sn.startswith("U") or "U-BAHN" in ln or "UBAHN" in ln:
        return "U-Bahn"
    if sn.startswith("M") and row["mode_code"] is None:  # many trams are Mxx
        return "Tram"
    return row["mode_code"]

routes["mode"] = routes.apply(name_mode, axis=1)

# Build stop<->mode pairs
trip_modes = feed.trips[["trip_id","route_id"]].merge(routes[["route_id","mode"]], on="route_id", how="left")
stops_served = feed.stop_times[["stop_id","trip_id"]].merge(trip_modes, on="trip_id", how="left")
stops_served = stops_served.dropna(subset=["mode"])

stop_mode_pairs = (stops_served.groupby(["stop_id","mode"], as_index=False)
                   .size().rename(columns={"size":"n"}))

print("Routes mapped by mode:\n", routes["mode"].value_counts(dropna=False))

Routes mapped by mode:
 mode
Bus       1194
S-Bahn      54
Tram        43
None        26
U-Bahn      13
Name: count, dtype: int64


In [14]:
import numpy as np
import geopandas as gpd
from scipy.spatial import cKDTree

# Clean GTFS stops and reproject to graph CRS
stops = (feed.stops[["stop_id","stop_lat","stop_lon"]]
         .dropna(subset=["stop_lat","stop_lon"]).copy())
stops = stops[
    np.isfinite(stops.stop_lat) & np.isfinite(stops.stop_lon) &
    stops.stop_lat.between(-90, 90) & stops.stop_lon.between(-180, 180)
]
stops["geometry"] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat, crs=4326)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326).to_crs(nodes_gdf.crs)
stops = stops[stops.geometry.notna() & ~stops.geometry.is_empty].copy()
stops = gpd.clip(stops, districts.unary_union)

# Merge with modes
stops_modes = stop_mode_pairs.merge(stops[["stop_id","geometry"]], on="stop_id", how="inner")
stops_modes = gpd.GeoDataFrame(stops_modes, geometry="geometry", crs=nodes_gdf.crs)

# KDTree snap to graph nodes (robust)
NX = nodes_gdf["x"].to_numpy(dtype=float)
NY = nodes_gdf["y"].to_numpy(dtype=float)
node_ids = nodes_gdf.index.to_numpy()
mask_nodes = np.isfinite(NX) & np.isfinite(NY)
tree = cKDTree(np.column_stack([NX[mask_nodes], NY[mask_nodes]]))
node_ids = node_ids[mask_nodes]

PX = stops_modes.geometry.x.to_numpy(dtype=float)
PY = stops_modes.geometry.y.to_numpy(dtype=float)
mask_pts = np.isfinite(PX) & np.isfinite(PY)
ii = tree.query(np.column_stack([PX[mask_pts], PY[mask_pts]]), k=1)[1]
stops_modes.loc[mask_pts, "node_id"] = node_ids[ii]

print("Snapped stops per mode:\n", stops_modes["mode"].value_counts())

Snapped stops per mode:
 mode
Bus       6544
Tram       840
U-Bahn     435
S-Bahn      77
Name: count, dtype: int64


In [15]:
def node_times_to_sources(G, source_nodes):
    if not source_nodes:
        return {}
    super_src = -1
    if super_src in G: G.remove_node(super_src)
    G.add_node(super_src)
    for n in source_nodes:
        G.add_edge(super_src, n, walk_time_s=0)
    return nx.single_source_dijkstra_path_length(G, super_src, weight="walk_time_s")

modes = ["S-Bahn", "U-Bahn", "Tram", "Bus"]
per_mode = {}

for m in modes:
    mode_nodes = stops_modes.loc[stops_modes["mode"] == m, "node_id"].dropna().unique().tolist()
    if not mode_nodes:
        per_mode[m] = pd.DataFrame({"district": [], m: []})
        print(f"Warning: no stop nodes for mode '{m}'")
        continue

    times = node_times_to_sources(Gp, mode_nodes)

    df = bld[["district","node_id"]].copy()
    df["walk_min"] = df["node_id"].map(lambda nid: (times.get(nid, np.nan) / 60.0))
    df = df.dropna(subset=["walk_min"])
    df = df[df["walk_min"] < MAX_WALK_MIN]  # cap outliers ≥ 30 min

    # MEDIAN aggregation per your spec
    per_mode[m] = (df.groupby("district")["walk_min"]
                     .median()
                     .reset_index()
                     .rename(columns={"walk_min": m}))

# Assemble 12×4
table = districts[["district"]].drop_duplicates().sort_values("district")
for m in modes:
    table = table.merge(per_mode[m], on="district", how="left")

table[["S-Bahn","U-Bahn","Tram","Bus"]] = table[["S-Bahn","U-Bahn","Tram","Bus"]].round(2)
table

,district,S-Bahn,U-Bahn,Tram,Bus
0,Charlottenburg-Wilmersdorf,21.76,14.39,27.61,7.78
1,Friedrichshain-Kreuzberg,24.20,13.24,8.52,6.27
2,Lichtenberg,23.47,20.02,12.44,10.33
3,Marzahn-Hellersdorf,22.65,23.30,15.83,9.12
4,Mitte,12.65,12.45,16.20,7.92
5,Neukölln,NaN,18.85,28.73,7.63
6,Pankow,20.34,20.10,15.00,9.91
7,Reinickendorf,NaN,18.78,24.85,9.15
8,Spandau,21.57,21.64,NaN,8.43
9,Steglitz-Zehlendorf,0.25,20.02,NaN,8.00


In [16]:
# Save to CSV on out folder
OUT_CSV = "average_walk_distances_per_mode.csv"
table.to_csv(OUT_CSV, index=False)  